# Point mutations in NTRKs

In [12]:
from openeye import oechem
from openforcefield.topology import Molecule

input_pdb = "../md/data/md_equil/4YNE/4YNE_prepped.pdb"
input_ligands_sdf = "../docking/data/4YNE_hybrid_docking.sdf"



In [69]:
# Load input SDF file containing all docked ligands
ifs = oechem.oemolistream(input_ligands_sdf)
ifs.SetFormat(oechem.OEFormat_SDF)

# Set up a dictionary for the new file names we will generate
ofs_dict = {
    "larotrectinib" : "4YNE_hybrid_docking_larotrectinib.sdf",
    "selitrectinib" : "4YNE_hybrid_docking_selitrectinib.sdf",
    "repotrectinib" : "4YNE_hybrid_docking_repotrectinib.sdf"
    }

# Loop through each molecule and create an SDF file containing just one ligand
for ofs_key, mol in zip(ofs_dict, ifs.GetOEMols()):
    
    ofs = oechem.oemolostream(ofs_dict[ofs_key])
    ofs.SetFormat(oechem.OEFormat_SDF)

    oechem.OEWriteMolecule(ofs, mol)